### Mounting drive on Colab

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effects of changing Batch Size



---


Experimental Setup:

Following are the details of the experimental setup for the above-mentioned experiment:

Model used: InceptionV3

Note: In order to make sure even though a range of batch sizes were used for this experiment all the models were trained for the same number of steps the epochs were changed accordingly.

For Ex: 

        Batch sizes used: [12, 24, 36, 48, 60, 72, 84, 96, 108, 120]

        Number of steps: 2000

        Given the relationship:

        Number of steps = (Dataset Size/ Batch Size) * Number of Epochs

        Therefore, if: Dataset Size = 24,000

        Batch Size = 12

        Number of Steps = 24,000/12 * epochs = 2,000 * epochs

        Number of Epochs = 1

        Dataset Size = 24,000

        Batch Size = 24

        Number of Steps = 24,000/24 * epochs = 1000 * epochs

        Number of Epochs = 2

        Therefore, Epochs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### For ImageNet as a source

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()


pre_trained_model = InceptionV3(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = 'None')

#for layer in pre_trained_model.layers:
#  layer.trainable = False

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
def compile_model():
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(1000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(500, activation='relu')(x)                  
  # Add a final softmax layer for classification
  x = layers.Dense(15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [ ]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [ ]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [ ]:
print("*********** Fitting model for 12 batch size and for 1 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 12, epochs=1, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 12 batch size and for 1 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


94/94 [==============================] - 10s 72ms/step - loss: 2.6754 - acc: 0.1790 - auc: 0.6849 - recall: 0.0520 - precision: 0.3570 - hamming_loss: 0.9480
***********test accuracy is [2.675435781478882, 0.17900000512599945, 0.684900164604187, 0.052000001072883606, 0.3569793999195099, 0.9480000138282776] ***************
******************* Train time is 131.09432462199993 ***********


In [ ]:
print("*********** Fitting model for 24 batch size and for 2 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 24, epochs=2, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 24 batch size and for 2 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
1000/1000 [==============================] - 69s 63ms/step - loss: 5.3503 - acc: 0.1498 - auc_1: 0.6118 - recall_1: 0.0636 - precision_1: 0.1813 - hamming_loss: 0.9364 - val_loss: 2.5739 - val_acc: 0.2040 - val_auc_1: 0.6992 - val_recall_1: 0.0517 - val_precision_1: 0.4178 - val_hamming_loss: 0.9483
Epoch 2/2
94/94 [==============================] - 8s 64ms/step - loss: 2.2941 - acc: 0.2597 - auc_1: 0.7673 - recall_1: 0.0897 - precision_1: 0.5558 - hamming_loss: 0.9103
***********test accuracy is [2.2941043376922607, 0.25966668128967285, 0.7672562599182129, 0.08966666460037231, 0.5557851195335388, 0.9103333353996277] ***************
******************* Train time is 260.03573512899993 ***********


In [ ]:
print("*********** Fitting model for 36 batch size and for 3 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 36, epochs=3, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 36 batch size and for 3 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/3
667/667 [==============================] - 57s 75ms/step - loss: 5.8006 - acc: 0.1479 - auc_2: 0.6059 - recall_2: 0.0657 - precision_2: 0.1744 - hamming_loss: 0.9343 - val_loss: 2.7748 - val_acc: 0.1770 - val_auc_2: 0.6703 - val_recall_2: 0.0467 - val_precision_2: 0.3024 - val_hamming_loss: 0.9533
Epoch 2/3
667/667 [==============================] - 46s 70ms/step - loss: 2.4525 - acc: 0.2376 - auc_2: 0.7344 - recall_2: 0.0834 - precision_2: 0.4649 - hamming_loss: 0.9166 - val_loss: 2.3821 - val_acc: 0.2603 - val_auc_2: 0.7507 - val_recall_2: 0.0960 - val_precision_2: 0.4668 - val_hamming_loss: 0.9040
Epoch 3/3
94/94 [==============================] - 6s 62ms/step - loss: 2.1088 - acc: 0.3290 - auc_2: 0.8077 - recall_2: 0.1397 - precision_2: 0.6537 - hamming_loss: 0.8603
***********test accuracy is [2.1088106632232666, 0.32899999618530273, 0.8077096939086914, 0.13966666162014008, 0.6536661386489868, 0.8603333234786987] ***************
******************* Train time is 410.1078

In [ ]:
print("*********** Fitting model for 48 batch size and for 4 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 48, epochs=4, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 48 batch size and for 4 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/4
500/500 [==============================] - 48s 81ms/step - loss: 6.5609 - acc: 0.1429 - auc_3: 0.5901 - recall_3: 0.0715 - precision_3: 0.1613 - hamming_loss: 0.9285 - val_loss: 2.6641 - val_acc: 0.1877 - val_auc_3: 0.6755 - val_recall_3: 0.0597 - val_precision_3: 0.4192 - val_hamming_loss: 0.9403
Epoch 2/4
500/500 [==============================] - 38s 77ms/step - loss: 2.5034 - acc: 0.2311 - auc_3: 0.7241 - recall_3: 0.0823 - precision_3: 0.4381 - hamming_loss: 0.9177 - val_loss: 2.5481 - val_acc: 0.2013 - val_auc_3: 0.7213 - val_recall_3: 0.0940 - val_precision_3: 0.3922 - val_hamming_loss: 0.9060
Epoch 3/4
500/500 [==============================] - 40s 80ms/step - loss: 2.2219 - acc: 0.3002 - auc_3: 0.7861 - recall_3: 0.1262 - precision_3: 0.5670 - hamming_loss: 0.8738 - val_loss: 2.4023 - val_acc: 0.2707 - val_auc_3: 0.7742 - val_recall_3: 0.1257 - val_precision_3: 0.4107 - val_hamming_loss: 0.8743
Epoch 4/4
94/94 [==============================] - 6s 62ms/step - loss: 2

In [ ]:
print("*********** Fitting model for 60 batch size and for 5 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 60, epochs=5, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 60 batch size and for 5 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/5
400/400 [==============================] - 43s 88ms/step - loss: 6.8029 - acc: 0.1428 - auc_4: 0.5917 - recall_4: 0.0742 - precision_4: 0.1619 - hamming_loss: 0.9258 - val_loss: 2.6760 - val_acc: 0.2110 - val_auc_4: 0.6877 - val_recall_4: 0.0527 - val_precision_4: 0.4158 - val_hamming_loss: 0.9473
Epoch 2/5
400/400 [==============================] - 33s 82ms/step - loss: 2.5284 - acc: 0.2292 - auc_4: 0.7208 - recall_4: 0.0818 - precision_4: 0.4408 - hamming_loss: 0.9182 - val_loss: 2.3595 - val_acc: 0.2637 - val_auc_4: 0.7602 - val_recall_4: 0.1027 - val_precision_4: 0.4881 - val_hamming_loss: 0.8973
Epoch 3/5
400/400 [==============================] - 33s 83ms/step - loss: 2.2523 - acc: 0.2962 - auc_4: 0.7811 - recall_4: 0.1225 - precision_4: 0.5487 - hamming_loss: 0.8775 - val_loss: 2.1843 - val_acc: 0.3177 - val_auc_4: 0.7901 - val_recall_4: 0.1317 - val_precision_4: 0.6270 - val_hamming_loss: 0.8683
Epoch 4/5
400/400 [==============================] - 34s 84ms/step - loss

In [ ]:
print("*********** Fitting model for 72 batch size and for 6 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 72, epochs=6, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 72 batch size and for 6 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/6
334/334 [==============================] - 44s 107ms/step - loss: 8.3091 - acc: 0.1332 - auc_5: 0.5777 - recall_5: 0.0816 - precision_5: 0.1467 - hamming_loss: 0.9184 - val_loss: 3.7809 - val_acc: 0.1037 - val_auc_5: 0.6003 - val_recall_5: 0.0670 - val_precision_5: 0.1368 - val_hamming_loss: 0.9330
Epoch 2/6
334/334 [==============================] - 32s 96ms/step - loss: 2.6201 - acc: 0.2167 - auc_5: 0.7065 - recall_5: 0.0816 - precision_5: 0.4016 - hamming_loss: 0.9184 - val_loss: 2.7102 - val_acc: 0.2203 - val_auc_5: 0.7026 - val_recall_5: 0.1017 - val_precision_5: 0.4230 - val_hamming_loss: 0.8983
Epoch 3/6
334/334 [==============================] - 32s 95ms/step - loss: 2.2744 - acc: 0.2871 - auc_5: 0.7735 - recall_5: 0.1201 - precision_5: 0.5668 - hamming_loss: 0.8799 - val_loss: 2.4853 - val_acc: 0.2523 - val_auc_5: 0.7443 - val_recall_5: 0.1113 - val_precision_5: 0.4108 - val_hamming_loss: 0.8887
Epoch 4/6
334/334 [==============================] - 32s 96ms/step - los

In [ ]:
print("*********** Fitting model for 84 batch size and for 7 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 84, epochs=7, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 84 batch size and for 7 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/7
286/286 [==============================] - 41s 116ms/step - loss: 9.1315 - acc: 0.1311 - auc_6: 0.5725 - recall_6: 0.0772 - precision_6: 0.1421 - hamming_loss: 0.9228 - val_loss: 2.7999 - val_acc: 0.1907 - val_auc_6: 0.6694 - val_recall_6: 0.0643 - val_precision_6: 0.3154 - val_hamming_loss: 0.9357
Epoch 2/7
286/286 [==============================] - 29s 102ms/step - loss: 2.6672 - acc: 0.2145 - auc_6: 0.6949 - recall_6: 0.0760 - precision_6: 0.4117 - hamming_loss: 0.9240 - val_loss: 2.5879 - val_acc: 0.2160 - val_auc_6: 0.7039 - val_recall_6: 0.0813 - val_precision_6: 0.4000 - val_hamming_loss: 0.9187
Epoch 3/7
286/286 [==============================] - 29s 100ms/step - loss: 2.3000 - acc: 0.2858 - auc_6: 0.7676 - recall_6: 0.1131 - precision_6: 0.5583 - hamming_loss: 0.8869 - val_loss: 2.4428 - val_acc: 0.2653 - val_auc_6: 0.7534 - val_recall_6: 0.1353 - val_precision_6: 0.4203 - val_hamming_loss: 0.8647
Epoch 4/7
286/286 [==============================] - 29s 100ms/step - 

In [ ]:
print("*********** Fitting model for 96 batch size and for 8 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 96, epochs=8, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 96 batch size and for 8 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/8
250/250 [==============================] - 36s 115ms/step - loss: 10.2414 - acc: 0.1310 - auc_7: 0.5704 - recall_7: 0.0827 - precision_7: 0.1404 - hamming_loss: 0.9173 - val_loss: 3.0359 - val_acc: 0.1637 - val_auc_7: 0.6487 - val_recall_7: 0.0650 - val_precision_7: 0.3071 - val_hamming_loss: 0.9350
Epoch 2/8
250/250 [==============================] - 27s 108ms/step - loss: 2.7309 - acc: 0.2097 - auc_7: 0.6918 - recall_7: 0.0799 - precision_7: 0.3747 - hamming_loss: 0.9201 - val_loss: 2.5505 - val_acc: 0.2397 - val_auc_7: 0.7172 - val_recall_7: 0.0843 - val_precision_7: 0.4210 - val_hamming_loss: 0.9157
Epoch 3/8
250/250 [==============================] - 29s 117ms/step - loss: 2.3317 - acc: 0.2774 - auc_7: 0.7627 - recall_7: 0.1147 - precision_7: 0.5436 - hamming_loss: 0.8852 - val_loss: 2.2306 - val_acc: 0.2773 - val_auc_7: 0.7841 - val_recall_7: 0.1027 - val_precision_7: 0.5714 - val_hamming_loss: 0.8973
Epoch 4/8
250/250 [==============================] - 27s 107ms/step -

In [ ]:
print("*********** Fitting model for 108 batch size and for 9 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 108, epochs=9, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 108 batch size and for 9 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/9
223/223 [==============================] - 37s 130ms/step - loss: 10.3571 - acc: 0.1268 - auc_8: 0.5635 - recall_8: 0.0866 - precision_8: 0.1366 - hamming_loss: 0.9134 - val_loss: 3.5022 - val_acc: 0.1637 - val_auc_8: 0.6224 - val_recall_8: 0.0877 - val_precision_8: 0.2400 - val_hamming_loss: 0.9123
Epoch 2/9
223/223 [==============================] - 26s 117ms/step - loss: 2.8480 - acc: 0.2050 - auc_8: 0.6812 - recall_8: 0.0847 - precision_8: 0.3609 - hamming_loss: 0.9153 - val_loss: 2.8981 - val_acc: 0.1840 - val_auc_8: 0.6869 - val_recall_8: 0.0967 - val_precision_8: 0.2606 - val_hamming_loss: 0.9033
Epoch 3/9
223/223 [==============================] - 27s 120ms/step - loss: 2.3790 - acc: 0.2773 - auc_8: 0.7555 - recall_8: 0.1165 - precision_8: 0.5169 - hamming_loss: 0.8835 - val_loss: 2.4104 - val_acc: 0.2730 - val_auc_8: 0.7541 - val_recall_8: 0.1083 - val_precision_8: 0.4829 - val_hamming_loss: 0.8917
Epoch 4/9
223/223 [==============================] - 26s 116ms/step -

In [ ]:
print("*********** Fitting model for 120 batch size and for 10 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 120 batch size and for 10 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 34s 132ms/step - loss: 11.5846 - acc: 0.1221 - auc_9: 0.5575 - recall_9: 0.0836 - precision_9: 0.1305 - hamming_loss: 0.9164 - val_loss: 3.0992 - val_acc: 0.1477 - val_auc_9: 0.6522 - val_recall_9: 0.0767 - val_precision_9: 0.2176 - val_hamming_loss: 0.9233
Epoch 2/10
200/200 [==============================] - 24s 122ms/step - loss: 2.9639 - acc: 0.1962 - auc_9: 0.6714 - recall_9: 0.0786 - precision_9: 0.3286 - hamming_loss: 0.9214 - val_loss: 2.6284 - val_acc: 0.2537 - val_auc_9: 0.7029 - val_recall_9: 0.0817 - val_precision_9: 0.4757 - val_hamming_loss: 0.9183
Epoch 3/10
200/200 [==============================] - 24s 122ms/step - loss: 2.4097 - acc: 0.2660 - auc_9: 0.7473 - recall_9: 0.1078 - precision_9: 0.5254 - hamming_loss: 0.8922 - val_loss: 2.4674 - val_acc: 0.2577 - val_auc_9: 0.7408 - val_recall_9: 0.0943 - val_precision_9: 0.5193 - val_hamming_loss: 0.9057
Epoch 4/10
200/200 [==============================] - 24s 122ms/st